In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.2 MB/s eta 0:00:00


In [35]:
import spacy
import transformers

# load the spaCy model for NER
nlp = spacy.load("en_core_web_sm")

# define the list of text to analyze
texts = [
   """filings.  
Providing remarks today are Ryals McMullian, president and CEO, and Steve Kinsey, our CFO. Ryals, I’ll turn it
over to you, President and CEO
We continued to execute well in the third quarter of 2022, driving quarterly sales to record levels. I am
particularly proud of our performance given the unique challenges presented by the current environment.
Recessionary economic forces and inflation remain significant factors, and I’ll discuss those topics in more
detail a little bit later in the call. """
]

# extract the persons from the text using spaCy NER
persons = []
for text in texts:
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            persons.append(ent.text)

# print the list of persons
print("List of persons:", persons)

# load the fill-mask pipeline from transformers library
fill_mask = transformers.pipeline("fill-mask")

# define the text to fill the mask
text = """Thank you. And now I’ll turn it back to Ryals. <mask> , CEO and president .Thank you, Steve."""

# get the most probable value from the persons list
most_probable_person = ""
max_score = 0.0

if "<mask>" not in text:
    print("Error: <mask> token not found in the input text.")
else:
    # get the most probable value from the persons list
    most_probable_person = ""
    max_score = 0.0
    for person in persons:
        filled_text = text.replace("<mask>", person)
        score = fill_mask(text)[0]["score"]
        if score > max_score:
            max_score = score
            most_probable_person = person

    if most_probable_person != "":
        # fill in the mask with the most probable person from the persons list
        filled_text = text.replace("<mask>", most_probable_person)
        print("Filled text:", filled_text)
    else:
        print("No person found in the text.")


No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


List of persons: ['Ryals McMullian', 'Steve Kinsey']
Filled text: Thank you. And now I’ll turn it back to Ryals. Ryals McMullian , CEO and president .Thank you, Steve.
